### Feature extraction
Following tutorial: https://towardsdatascience.com/understanding-feature-engineering-part-4-deep-learning-methods-for-text-data-96c44370bbfa

In [8]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 200
%matplotlib inline

### Toy Corpus

In [9]:
corpus = [
    'Ich kann keine Bilder öffnen.',
    'Der Doppelklick auf Bilder funktioniert nicht mehr.',
    'In Buch Nr. 712 auf Seite 12 wurde ein Wort falsch geschrieben!',
    'Der Buchtitel wurde falsch geschrieben. NLKT wurde geschrieben anstatt NLTK!',
    'Ich kann die gekaufte Lizenz nicht mehr aktivieren. Was mache ich falsch?',
    'Die Lizenz funktioniert nicht!',
    'Ich kann mich nicht mehr einloggen.',
    'Mein Passwort scheint nicht mehr zu funktionieren!',
    'Der Betrag wurde von meiner Kreditkarte abgezogen, das Buch habe ich immernoch nicht erhalten!',
    'Die Bezahlung wurde doppelt von meinem Konto abgebucht. #@!?$!!!'    
]

labels = ['bug', 'bug', 'grammar', 'grammar', 'license', 'license', 'login', 'login', 'payment', 'payment']

corpus = np.array(corpus)
corpus_df = pd.DataFrame(
    {
        'Document': corpus, 
        'Category': labels
    }
)

corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,Ich kann keine Bilder öffnen.,bug
1,Der Doppelklick auf Bilder funktioniert nicht mehr.,bug
2,In Buch Nr. 712 auf Seite 12 wurde ein Wort falsch geschrieben!,grammar
3,Der Buchtitel wurde falsch geschrieben. NLKT wurde geschrieben anstatt NLTK!,grammar
4,Ich kann die gekaufte Lizenz nicht mehr aktivieren. Was mache ich falsch?,license
5,Die Lizenz funktioniert nicht!,license
6,Ich kann mich nicht mehr einloggen.,login
7,Mein Passwort scheint nicht mehr zu funktionieren!,login
8,"Der Betrag wurde von meiner Kreditkarte abgezogen, das Buch habe ich immernoch nicht erhalten!",payment
9,Die Bezahlung wurde doppelt von meinem Konto abgebucht. #@!?$!!!,payment


### Text pre-processing

In [10]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('german')

def normalize_document(doc):
    # lower case and remove special characters \ whitespaces
    doc = re.sub(r'[^\u00C0-\u017Fa-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    
    # tokenize document
    tokens = wpt.tokenize(doc)
    
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [11]:
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['bilder öffnen', 'doppelklick bilder funktioniert mehr',
       'buch nr seite wurde wort falsch geschrieben',
       'buchtitel wurde falsch geschrieben nlkt wurde geschrieben anstatt nltk',
       'gekaufte lizenz mehr aktivieren mache falsch',
       'lizenz funktioniert', 'mehr einloggen',
       'passwort scheint mehr funktionieren',
       'betrag wurde kreditkarte abgezogen buch immernoch erhalten',
       'bezahlung wurde doppelt konto abgebucht'], dtype='<U70')

### Skip-gram Model

In [12]:
from keras.preprocessing import text

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_corpus)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1 
embed_size = 100

wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in norm_corpus]
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Using TensorFlow backend.


Vocabulary Size: 34
Vocabulary Sample: [('wurde', 1), ('mehr', 2), ('falsch', 3), ('geschrieben', 4), ('bilder', 5), ('funktioniert', 6), ('buch', 7), ('lizenz', 8), ('öffnen', 9), ('doppelklick', 10)]


In [13]:
from keras.preprocessing.sequence import skipgrams

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]
len_skip_grams = len(skip_grams[1][0])
print(f'valid range for (skip_grams[1][0], skip_grams[1][1]) is {len_skip_grams}\n')

# view sample skip-grams
pairs, labels = skip_grams[1][0], skip_grams[1][1]
for i in range(len_skip_grams):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
        id2word[pairs[i][0]], pairs[i][0], 
        id2word[pairs[i][1]], pairs[i][1], 
        labels[i])
     )

valid range for (skip_grams[1][0], skip_grams[1][1]) is 24

(mehr (2), anstatt (16)) -> 0
(mehr (2), passwort (22)) -> 0
(mehr (2), funktioniert (6)) -> 1
(funktioniert (6), mehr (2)) -> 1
(doppelklick (10), aktivieren (19)) -> 0
(doppelklick (10), funktioniert (6)) -> 1
(mehr (2), doppelklick (10)) -> 1
(funktioniert (6), doppelklick (10)) -> 1
(funktioniert (6), nltk (17)) -> 0
(bilder (5), doppelklick (10)) -> 1
(funktioniert (6), nlkt (15)) -> 0
(doppelklick (10), nltk (17)) -> 0
(bilder (5), abgezogen (27)) -> 0
(mehr (2), bilder (5)) -> 1
(doppelklick (10), immernoch (28)) -> 0
(funktioniert (6), bilder (5)) -> 1
(bilder (5), aktivieren (19)) -> 0
(funktioniert (6), funktionieren (24)) -> 0
(doppelklick (10), bilder (5)) -> 1
(bilder (5), mehr (2)) -> 1
(mehr (2), konto (32)) -> 0
(bilder (5), funktioniert (6)) -> 1
(bilder (5), nltk (17)) -> 0
(doppelklick (10), mehr (2)) -> 1
